In [22]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import nilearn.image, nilearn.masking, nilearn.plotting
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [23]:
data_dir = '/Volumes/clmnlab/GA/MVPA/LSS_pb02_MO_short_duration/data/'

In [24]:
def fast_masking(img, roi):
    img_data = img.get_data()
    roi_mask = roi.get_data().astype(bool)
    
    if img_data.shape[:3] != roi_mask.shape:
        raise ValueError('different shape while masking! img=%s and roi=%s' % (img_data.shape, roi_mask.shape))
        
    return img_data[roi_mask, :].T

In [25]:
def accuracy(train_X, train_y, test_X, test_y):
    lda.fit(train_X, train_y)
    return np.mean(lda.predict(test_X) == test_y)

def cross_validation(datasets, target):
    train_y = target + target
    test_y = target
    
    result = []
    
    for i in range(len(datasets)):
        train_X = np.concatenate([a for j, a in enumerate(datasets) if j != i])
        test_X = datasets[i]
        
        result.append(accuracy(train_X, train_y, test_X, test_y))
        
    return np.mean(result)

In [26]:
roi_dir = '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/'

dmn_list = [
    5, 6, 11, 12, 13, 14, 27, 28, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 51, 52, 75, 76, 79, 80, 81, 82, 83, 84, 87, 88, 95, 96, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 143, 144, 151, 152, 153, 154, 175, 176, 181, 182, 187, 188
]

motor_list = [
    53, 54, 55, 56, 57, 58, 59, 60, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164
]

visual_list = [
    105, 106, 119, 120, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210
]


In [27]:
module = visual_list

roi_path = []

for m in module:
    roi_path.append(roi_dir + 'fan.roi.resam.%03d.nii' % (m))
#roi_path = sorted(glob.glob(roi_dir + '*.nii'))

roi_imgs = []

for path in roi_path:
    roi_imgs.append(nilearn.image.load_img(path))

In [28]:
roi_path

['/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.105.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.106.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.119.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.120.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.189.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.190.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.191.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.192.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.193.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.194.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.195.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.196.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fan.roi.resam.197.nii',
 '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/fa

In [29]:
subj_list = [
    'GA01', 'GA02', 'GA05', 'GA07', 'GA08', 'GA11', 'GA12', 'GA13', 'GA14', 'GA15',
    'GA18', 'GA19', 'GA20', 'GA21', 'GA23', 'GA26', 'GA27', 'GA28', 'GA29', 'GA30',
    'GA31', 'GA32', 'GA33', 'GA34', 'GA35', 'GA36', 'GA37', 'GA38', 'GA42', 'GA44'
]
    
#    'GB01', 'GB02', 'GB05', 'GB07', 'GB08', 'GB11', 'GB12', 'GB13', 'GB14', 'GB15',
#    'GB18', 'GB19', 'GB20', 'GB21', 'GB23', 'GB26', 'GB27', 'GB28', 'GB29', 'GB30',
#    'GB31', 'GB32', 'GB33', 'GB34', 'GB35', 'GB36', 'GB37', 'GB38', 'GB42', 'GB44'
#]

In [30]:
data = {}

for subj in subj_list:
    for run in range(1, 7):
        data[subj, run] = nilearn.image.load_img(data_dir + 'betasLSS.MO.shortdur.%s.r%02d.nii.gz' % (subj, run))
    print(subj, end='\r')

In [31]:
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

In [32]:
target_path = list(range(1, 13)) * 8
target_pos = []

with open('/Volumes/clmnlab/GA/MVPA/LSS_pb02/behaviors/targetID.txt') as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]

In [33]:
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

results = {}
 
for subj in subj_list:
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        
        if ('pos', 'unpracticed', subj, roi_index) in results:
            continue
        
        X = [
            fast_masking(roi=roi, img=data[subj, 1]), 
            fast_masking(roi=roi, img=data[subj, 2]), 
            fast_masking(roi=roi, img=data[subj, 3]), 
        ]
        
        results['pos', 'practiced', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'practiced', subj, roi_index] = cross_validation(X, target_path)
        
        X = [
            fast_masking(roi=roi, img=data[subj, 4]), 
            fast_masking(roi=roi, img=data[subj, 5]), 
            fast_masking(roi=roi, img=data[subj, 6]), 
        ]
        
        results['pos', 'unpracticed', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'unpracticed', subj, roi_index] = cross_validation(X, target_path)
        
        print(subj, roi_index, end='\r')

In [34]:
len(results)

3120

In [35]:
results

{('pos', 'practiced', 'GA01', 1): 0.4930555555555555,
 ('path', 'practiced', 'GA01', 1): 0.2881944444444444,
 ('pos', 'unpracticed', 'GA01', 1): 0.40625,
 ('path', 'unpracticed', 'GA01', 1): 0.2152777777777778,
 ('pos', 'practiced', 'GA01', 2): 0.48958333333333326,
 ('path', 'practiced', 'GA01', 2): 0.2673611111111111,
 ('pos', 'unpracticed', 'GA01', 2): 0.3125,
 ('path', 'unpracticed', 'GA01', 2): 0.19791666666666666,
 ('pos', 'practiced', 'GA01', 3): 0.2916666666666667,
 ('path', 'practiced', 'GA01', 3): 0.10416666666666667,
 ('pos', 'unpracticed', 'GA01', 3): 0.28125,
 ('path', 'unpracticed', 'GA01', 3): 0.09722222222222221,
 ('pos', 'practiced', 'GA01', 4): 0.2986111111111111,
 ('path', 'practiced', 'GA01', 4): 0.0763888888888889,
 ('pos', 'unpracticed', 'GA01', 4): 0.2673611111111111,
 ('path', 'unpracticed', 'GA01', 4): 0.10416666666666667,
 ('pos', 'practiced', 'GA01', 5): 0.6145833333333334,
 ('path', 'practiced', 'GA01', 5): 0.6284722222222222,
 ('pos', 'unpracticed', 'GA01', 

In [36]:
data = []

In [37]:
subj_list = [
#    'GA01', 'GA02', 'GA05', 'GA07', 'GA08', 'GA11', 'GA12', 'GA13', 'GA14', 'GA15',
#    'GA18', 'GA19', 'GA20', 'GA21', 'GA23', 'GA26', 'GA27', 'GA28', 'GA29', 'GA30',
#    'GA31', 'GA32', 'GA33', 'GA34', 'GA35', 'GA36', 'GA37', 'GA38', 'GA42', 'GA44'
#]
    
   'GB01', 'GB02', 'GB05', 'GB07', 'GB08', 'GB11', 'GB12', 'GB13', 'GB14', 'GB15',
   'GB18', 'GB19', 'GB20', 'GB21', 'GB23', 'GB26', 'GB27', 'GB28', 'GB29', 'GB30',
   'GB31', 'GB32', 'GB33', 'GB34', 'GB35', 'GB36', 'GB37', 'GB38', 'GB42', 'GB44'
]

data = {}

for subj in subj_list:
    for run in range(1, 7):
        data[subj, run] = nilearn.image.load_img(data_dir + 'betasLSS.MO.shortdur.%s.r%02d.nii.gz' % (subj, run))
    print(subj, end='\r')

In [38]:
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

In [39]:
target_path = list(range(1, 13)) * 8
target_pos = []

with open('/Volumes/clmnlab/GA/MVPA/LSS_pb02/behaviors/targetID.txt') as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]

In [40]:
for subj in subj_list:
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        
        if ('pos', 'unpracticed', subj, roi_index) in results:
            continue
        
        X = [
            fast_masking(roi=roi, img=data[subj, 1]), 
            fast_masking(roi=roi, img=data[subj, 2]), 
            fast_masking(roi=roi, img=data[subj, 3]), 
        ]
        
        results['pos', 'practiced', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'practiced', subj, roi_index] = cross_validation(X, target_path)
        
        X = [
            fast_masking(roi=roi, img=data[subj, 4]), 
            fast_masking(roi=roi, img=data[subj, 5]), 
            fast_masking(roi=roi, img=data[subj, 6]), 
        ]
        
        results['pos', 'unpracticed', subj, roi_index] = cross_validation(X, target_pos)
        results['path', 'unpracticed', subj, roi_index] = cross_validation(X, target_path)
        
        print(subj, roi_index, end='\r')

In [41]:
df = pd.DataFrame(columns=['target', 'practiced', 'subject', 'roi', 'accu'])
for i,j in zip(results.keys(), results.values()):
    df = df.append({'target':i[0], 'practiced':i[1], 'subject':i[2], 'roi':i[3], 'accu':j}, ignore_index=True)
    
df.head()

,target,practiced,subject,roi,accu
0,pos,practiced,GA01,1,0.493056
1,path,practiced,GA01,1,0.288194
2,pos,unpracticed,GA01,1,0.406250
3,path,unpracticed,GA01,1,0.215278
4,pos,practiced,GA01,2,0.489583


In [42]:
df.to_csv("20190911_FAN_VISUAL_lda.csv")